# Assignment 2 - Elementary Probability and Information Theory 
# Boise State University NLP - Dr. Kennington

### Instructions and Hints:

* This notebook loads some data into a `pandas` dataframe, then does a small amount of preprocessing. Make sure your data can load by stepping through all of the cells up until question 1. 
* Most of the questions require you to write some code. In many cases, you will write some kind of probability function like we did in class using the data. 
* Some of the questions only require you to write answers, so be sure to change the cell type to markdown or raw text
* Don't worry about normalizing the text this time (e.g., lowercase, etc.). Just focus on probabilies. 
* Most questions can be answered in a single cell, but you can make as many additional cells as you need. 
* Follow the instructions on the corresponding assignment Trello card for submitting your assignment. 

In [1]:
import pandas as pd 

data = pd.read_csv('pnp-train.txt',delimiter='\t',encoding='latin-1', # utf8 encoding didn't work for this
                  names=['type','name']) # supply the column names for the dataframe

# this next line creates a new column with the lower-cased first word
data['first_word'] = data['name'].map(lambda x: x.lower().split()[0])

In [2]:
data[:10]

,type,name,first_word
0,drug,Dilotab,dilotab
1,movie,Beastie Boys: Live in Glasgow,beastie
2,person,Michelle Ford-Eriksson,michelle
3,place,Ramsbury,ramsbury
4,place,Market Bosworth,market
5,drug,Cyanide Antidote Package,cyanide
6,person,Bill Johnson,bill
7,place,Ettalong,ettalong
8,movie,The Suicide Club,the
9,place,Pézenas,pézenas


In [3]:
data.describe()

,type,name,first_word
count,21001,21001,21001
unique,5,20992,13703
top,movie,Epinal,the
freq,6262,2,635


## 1. Write a probability function/distribution $P(T)$ over the types. 

Hints:

* The Counter library might be useful: `from collections import Counter`
* Write a function `def P(T='')` that returns the probability of the specific value for T
* You can access the types from the dataframe by calling `data['type']`

In [39]:
from collections import Counter
type_counts = Counter(data['type'])
firstword_counts = Counter(data['first_word'])
def P(T=''):
    return type_counts[T] / sum(type_counts.values())

## 2. What is `P(T='movie')` ?

In [40]:
P(T='movie')

0.29817627732012764

## 3. Show that your probability distribution sums to one.

In [41]:
sum(float(P(t)) for t in Counter(data['type']).keys())

1.0

## 4. Write a joint distribution using the type and the first word of the name

Hints:

* The function is $P2(T,W_1)$
* You will need to count up types AND the first words, for example: ('person','bill)
* Using the [itertools.product](https://docs.python.org/2/library/itertools.html#itertools.product) function was useful for me here

In [31]:
def P2(T, W1):
    return data[(data['type'] == T) & (data['first_word'] == W1)].shape[0] / data.shape[0]

## 5. What is P2(T='person', W1='bill')? What about P2(T='movie',W1='the')?

In [32]:
P2(T='person', W1='bill')

0.00047616780153326033

In [33]:
P2(T='movie', W1='the')

0.02747488214846912

## 6. Show that your probability distribution P(T,W1) sums to one.

In [42]:
total_prob = sum(P2(T=type, W1=word) for type in type_counts.keys() for word in firstword_counts.keys())
print(total_prob)

1.0


## 7. Make a new function Q(T) from marginalizing over P(T,W1) and make sure that Q(T) sums to one.

Hints:

* Your Q function will call P(T,W1)
* Your check for the sum to one should be the same answer as Question 3, only it calls Q instead of P.

In [43]:
def Q(T):
    return sum(P2(T=T, W1=word) for word in firstword_counts.keys())

In [44]:
Q('movie')

0.29817627732012764

In [45]:
sum(Q(T=type) for type in data['type'].unique())

1.0

## 8. What is the KL Divergence of your Q function and your P function for Question 1?

* Even if you know the answer, you still need to write code that computes it.

In [55]:
# https://www.geeksforgeeks.org/floating-point-error-in-python/

from math import log
total_kl_divergence = sum((P(T=elem) * log(P(T=elem)/Q(T=elem))) for elem in type_counts.keys())

print("{:.1f}".format(total_kl_divergence))

-0.0


## 9. Convert from P(T,W1) to P(W1|T) 

Hints:

* Just write a comment cell, no code this time. 
* Note that $P(T,W1) = P(W1,T)$

P(W1|T) = (P(T,W1))/P(T)

## 10. Write a function `Pwt` (that calls the functions you already have) to compute $P(W_1|T)$.

* This will be something like the multiplication rule, but you may need to change something

In [14]:
def Pwt(W1='', T=''):
    return P2(T=T, W1=W1) / P(T)

## 11. What is P(W1='the'|T='movie')?

In [15]:
Pwt(W1='the',T='movie')

0.09214308527626956

## 12. Use Baye's rule to convert from P(W1|T) to P(T|W1). Write a function Ptw to reflect this. 

Hints:

* Call your other functions.
* You may need to write a function for P(W1) and you may need a new counter for `data['first_word']`

P(T|W1) = (P(T,W1))/P(W1)

In [16]:
def Ptw(T='',W1=''):
    word_counts = Counter(data['first_word'])
    return P2(T=T,W1=W1) / (word_counts[W1] / sum(word_counts.values()))

## 13 
### What is P(T='movie'|W1='the')? 
### What about P(T='person'|W1='the')?
### What about P(T='drug'|W1='the')?
### What about P(T='place'|W1='the')
### What about P(T='company'|W1='the')

In [17]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [18]:
Ptw(T='person',W1='the')

0.0

In [19]:
Ptw(T='drug',W1='the')

0.0

In [20]:
Ptw(T='place',W1='the')

0.0015748031496062994

In [21]:
Ptw(T='company',W1='the')

0.08976377952755905

## 14 Given this, if the word 'the' is found in a name, what is the most likely type?

Given the data above, if the word is 'the,' the type is most likely 'movie.'

In [25]:
def likely_type(W1=''):
    curr_type = ''
    highest_prob = 0.0
    for elem in data['type'].unique():
        temp_num = Ptw(T=elem, W1=W1)
        if (temp_num > highest_prob):
            highest_prob = temp_num
            curr_type = elem
    return curr_type
likely_type('the')

'movie'

## 15. Is Ptw(T='movie'|W1='the') the same as Pwt(W1='the'|T='movie') the same? Why or why not?

In [26]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [27]:
Pwt(W1='the', T='movie')

0.09214308527626956

They are different because the first statement is asking: what is the probability of 'movie' being the type given that the first word is 'the.'  The second statement is asking: what is the probability of 'the' being the first_word given that the type is 'movie.'

## 16. Do you think modeling Ptw(T|W1) would be better with a continuous function like a Gaussian? Why or why not?

- Answer in a markdown cell


No, modeling Ptw(T|W1) would not be better with a continuous function like a Gaussian because both 'type' and 'word' are both discrete variables in this case.  We would measure something like height or distance with a normal curve.

https://www.statisticshowto.com/probability-and-statistics/statistics-definitions/discrete-vs-continuous-variables/